<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Recherche d'Information et traitement de données massives </center>
    <center> Lab 4 : Evaluation des systèmes de recherche d'information </center>
<hr style=" border:none; height:3px;">
</h1>



L'objectif de cette séance est de mettre en oeuvre les différentes approches vues en cours pour évaluer les systèmes de recherche d'information.  La première partie du Lab consistera à évaluer et comparer les différents modèles mis en oeuvre dans le LAB 2 et le LAB 3, avec différents paramètres, sur la collection TIME. La deuxième partie, quant à elle, correspond à des exercices d'application disponibles sur EDUNAO (**à faire en dehors de la séance du LAB**).
 

## AVANT-PROPOS

Ce Lab clôture le travail pratique de la première partie du cours concernant **les fondements de la Recherche AD Hoc d'information** ainsi que leur application. Plus particulièrement, leur application s'est faite au travers de la création de moteurs de recherche sur la collection `TIME` et aujourd'hui nous clôturerons leur application par leur evaluation, toujours sur la collection `TIME`. Récapitulons les notions vues et que nous allons voir :
+ **Lab 1** : Notions clés de l'étape d'indexation : pré-traitements linguistique sur la collection (**Segmentation, Filtrage, Normalisation**), extraction du **vocabulaire d'indexation** et construction de l'**index inversé**.
+ **Lab 2** : Les premiers modèles de recherche de base en RI : **modèle booléen**, **modèle vectoriel** ;
+ **Lab 3**: les **modèles de recherche  probabilistes** et, 
+ **Lab 4** : l'**Evaluation** des systèmes de RI.

Ces différents Labs sont donc fortement dépendants. Pour faciliter votre travail, le répertoire concernant le LAB 4 comprend :
+ ce fichier qui est donc l'énoncé du LAB.
+ un répertoire [Data](./Data) qui contient le répertoire [Time](./Data/TIME) avec l'ensemble des fichiers de la collection.
+ un répertoire [Index](./Index) qui contient différents index inversés de la collection `TIME` et plus particulièrement :
    + le fichier [index_document](./Index/index_document) qui correspond à l'index de documents de la collection TIME écrit suivant le format décrit dans le LAB 1 et construit avec :
        + une segmentation à l'aide du `RegExpTokenizer` de la bibliothèque `nltk` ;
        + un filtrage à l'aide de la liste des stop-words fournis dans la collection `TIME` ;
        + une étape de normalisation par lemmatisation à l'aide du lemmatiseur `WordNetLemmatizer` de la bibliothèque `nltk` ;
        
    + le fichier [index_document.pickle](./Index/index_document.pickle) qui correspond au même index de documents que le précédent mais sauvegardé avec le module [`pickle`](https://www.datacamp.com/community/tutorials/pickle-python-tutorial) de python ;
    + le fichier [index_frequence.pickle](./Index/index_frequence.pickle) qui correspond à l'index de fréquence de la collection TIME (avec la même chaîne de traitement que précédemment) ;
    + le fichier [index_position.pickle](./Index/index_position.pickle) qui correspond à l'index de position de la collection TIME (avec la même chaîne de traitement que précédemment à l'exception de l'étape de segmentation qui a été faite avec le tokenizer `word_tokenize` de `nltk` pour respecter l'ordre d'apparition des tokens dans les documents ;
+ un répertoire [Utils](./Utils) qui contient les fichiers [Lab1.py](./Utils/Lab1.py) et [Lab2.py](./Utils/Lab2.py) qui contiennent l'ensemble des fonctions utiles des Lab 1 et Lab 2 et pouvant vous servir pour le Lab 4 d'aujourd'hui.
+ une archive nommée Archive_Lab4.zip permettant l'importation des fonctions fournies et du répertoire Utils pour le cas où vous effectuez votre lab sur colab.
    
**IL EST DONC IMPORTANT POUR LA SUITE DE BIEN STRUCTURER VOTRE PROJET ET DE BIEN RESPECTER CETTE STRUCTURE. IL FAUDRA DONC BIEN TRAVAILLER DANS UN REPERTOIRE DANS LEQUEL VOUS AUREZ VOTRE FICHIER `.py` (editeur python) OU `.ipynb`(notebook) ET LES REPERTOIRES `DATA`, `INDEX` ET `UTILS` AINSI QUE LES FICHIERS ASSOCIÉS.**

Enfin, voici à nouveau les **instructions pour permettre l'importation des fonctions fournies et du répertoire Utils pour le cas du lab effectué sur colab :**

1.    Dans le dossier Lab4_Evaluation téléchargé depuis git, vous trouverez une archive Archive_Lab4.zip.
2.   Ouvrir le panneau Fichiers de colab (c'est à dire cliquez sur le logo en forme de dossier à gauche) et cliquez sur le premier bouton en haut à gauche vous permettant de télécharger. Téléchargez ainsi l'Archive_Lab4.zip dans votre colab. Si besoin actualiser jusqu'à voir l'archive apparaître dans le panneau. 
3.    Exécutez les deux céllules de code suivantes pour déziper l'archive et installer à nouveau les librairies `nltk` et [`tt`](http://tt.brianwel.ch/en/latest/index.html). 




In [ ]:
!unzip Archive_lab4.zip

In [ ]:
!pip install nltk
import nltk
nltk.download()

In [ ]:
!pip install ttable

## PARTIE 1 : Collection TIME

Dans cette partie, nous allons comparer les différents modèles de recherche sur la collection TIME. Nous avons en effet dans la collection un ensemble de requêtes et de jugements de pertinence exhaustifs sur ces requêtes au travers des fichiers fournis. En particulier :
+ Le fichier [TIME.QUE](./Data/Time/TIME.QUE) qui contient un ensemble de requêtes exprimées en langage naturel. Chaque requête est identifiée dans le fichier par la chaîne de caractères ` *FIND      ID`
+ Le fichier [TIME.REL](./Data/Time/TIME.REL) qui contient les jugements de pertinence exaustifs pour chaque requête. Par exemple la ligne `1  268 288 304 308 323 326 334` doit être interprétée de la manière suivante : les documents pertinents de la collection pour la requête `1` sont les documents `268 288 304 308 323 32` et `334`.



### Les différents modèles

Nous allons comparer les modèles ci-dessous :
+ Le modèle booléen.
+ Le modèle vectoriel avec différents schémas de pondération :
    + La pondération `tf`.
    + La pondération `tf-idf`.
    + La pondération `tf-idf` normalisée.
    + La pondération `tf-idf` logarithmique.
    + La pondération `tf-idf` logarithmique normalisée.
+ Le modèle probabiliste BIM.
+ Le modèle probabiliste OKAPI BM 25.

Votre premier travail consiste donc, sur la base des travaux faits dans les LAB 1 et LAB 2, ou des corrections fournies, d'écrire les fonctions permettant d'appliquer ces différents modèles aux requêtes du fichier [TIME.QUE](./Data/Time/TIME.QUE).



#### Première étape : récupération des index inversés.

Dans cette première étape, nous allons récupérer, en les chargeant, les index inversés résultant de l'indexation de la collection `TIME` faite dans le [LAB 1](../Lab1_InvertedIndex/Lab1_Indexation-Student-Correction.ipynb). 

**A. Chargement de l'index de documents**

Vous pouvez pour cela utiliser la fonction `load_inverted_index_pickle(filename)` du fichier `Lab1.py` comme fait dans le code ci-dessous. 

In [ ]:
from Utils.Lab1 import load_inverted_index_pickle

# Recupération de l'index de documents
index_document = load_inverted_index_pickle("./Index/index_document.pickle")

**B. Chargement de l'index de fréquence**

1- Charger l'index de fréquence. Pour faciliter la suite des traitements cet index sera référencé par une variable nommée `index_frequence`.

In [ ]:
index_frequence = # A completer

**C. Chargement de l'index de position**

2- Charger l'index de position. Pour faciliter la suite des traitements cet index sera référencé par une variable nommée `index_position`.

In [ ]:
index_position = # A completer

#### Deuxième  étape : récupération des requêtes.

Votre premier travail consiste à charger le fichier [TIME.QUE](./Data/Time/TIME.QUE) et à parser ce fichier pour récupérer l'ensemble des requêtes que nous utiliserons pour l'évaluation. Comme pour la représentation de la Collection faite dans le LAB 1, on representera cet ensemble de requêtes sous la forme d'un dictionnaire qui a pour clé l'identifiant de la requête (l'`ID` dans la chaîne ` *FIND      ID`) et comme valeur la chaîne de caractères correspondant à la requête en langage naturel.

3- Ecrire la fonction `load_requests()` permettant de récupérer les requêtes. Pour cette étape, il vous est fortement conseillé d'utiliser la même approche que celle développée pour la récupération de la collection dans le Lab 1 et l'utilisation d'expressions régulières.


In [ ]:
import re

def load_requests(filename):
    requests = {}
    # A completer
    return requests


4- Appliquer cette fonction au fichier [TIME.QUE](./Data/Time/TIME.QUE).

In [ ]:
requests =# A completer


Voici une capture d'écran d'un morceau du résultat :


<img src="./Figures/requests.png" width="600" height="600" />

5- Appliquer à chaque requête la même chaîne de traitements linguistiques que ceux appliqués à la collection (segmentation, filtrage et normalisation). Pensez à utiliser les fonctions du Lab 1.

In [ ]:
from Utils.Lab1 import tokenize_Regexp_corpus,remove_stop_words,load_stop_word,collection_lemmatize

tokenized_requests= # A compléter
filtered_requests=# A compléter
lemmatized_requests=# A compléter

#### Troisième  étape : récupération des jugements de pertinence.

6- Vous allez maintenant récupérer les jugements de pertinence associés à chaque requête en parsant le fichier [TIME.REL](./Data/Time/TIME.REL). Ces jugements de pertinence seront eux aussi stockés sous la forme d'un dictionnaire avec comme **clé** l'`ID`de la requête et comme **valeur** une liste des documents jugés pertinents pour cette requête. Par exemple, la ligne `1  268 288 304 308 323 326 334` du fichier devra donc être représentée comme ceci dans le dictionnaire `{'1': ['268','288', '304', '308', '323', '326', '334' ],...}`.

In [ ]:
def load_relevance_judgments(filename):
    relevance_judgments = {}
    # A completer
    return relevance_judgments

7- Chargez et récupérez les jugements de pertinence de la collection TIME.

In [ ]:
# Test
relevance_judgments = #a compléter

Voici une capture d'écran d'un morceau du résultat :


<img src="./Figures/jugement.png" width="600" height="600" />

### EVALUATION DE RÉSULTATS DE RECHERCHE NON ORDONNÉS
### MODÈLE BOOLÉEN

Le modèle booléen est un modèle qui retourne des résultats de recherche non ordonnés. Les métriques que nous pouvons utiliser pour évaluer le système sont donc **le rappel**, **la précision** ou encore la **F-mesure** comme vues en cours et que nous pouvons calculer pour chaque requête ou moyenner sur l'ensemble des requêtes. Le principe pour évaluer le modèle est donc celui expliqué en cours (Cours 4 : évaluation - slide 28).

Pour chaque requête :
+ Exécuter le modèle sur la collection et récupérer les documents renvoyés par le système.
+ Marquer les documents pertinents en comparant avec les jugements de pertinence.
+ Calculer le rappel et la précision.

On sauvegardera les résultats de l'évaluation sous la forme d'un dictionnaire avec comme clé l'`ID` de la requête et comme valeur un dictionnaire avec une clé par type de mesure et en valeur le résultat de la mesure calculé sur les résultats de la requête. On calculera en particulier pour chaque requête :
+ son rappel (clé "recall")
+ sa précision (clé "precision")
+ la mesure F_1 (moyenne harmonique simple, c.f. slide 29 du cours 4)
+ la mesure F avec un paramètre $\beta$ de votre choix (c.f. slide 29 du cours 4)


8- A partir des différentes fonctions du LAB 2, écrire le code permettant d'évaluer le modèle booléen sur la collection `TIME`. En particulier il sera utile de :

a. Ecrire une fonction `transformation_lem_query_to_boolean(query,index,operator)` pour
+ **Transformer la requête sous sa forme booléenne** (rappel sur les expression régulières pouvant être utile : [ici](https://python.doctor/page-expressions-regulieres-regular-python)) et la transformer **en sa notation polonaise inversée** pour pouvoir appliquer les fonctions du LAB 2. On envisagera plusieurs cas :
    + Transformation de la requête en requête conjonctive, i.e. on considère que l'ensemble des termes de la requête sont reliés par des `AND`.
    + Transformation de la requête en requête disjonctive, i.e. on considère que l'ensemble des termes de la requête sont reliés par des `OR`.
+ **Vérifier que les termes de la requête sont bien des termes du vocabulaire d'index et les supprimer** ou les remplacer par leur mot le plus proche dans le vocabulaire **si ce n'est pas le cas. Ici, on se contentera de les supprimer des termes de la requête.**
+ **Supprimer (par simplicité) les mots de type "U.S" de la requête** car ils ne sont pas supportés par la bibliothèque `tt` utilisée pour implémenter le modèle booléen, de même que les tokens de type nombre.

b. Ecrire une fonction `apply_boolean_model(requests, collection_index,BooleanOperator)` pour appliquer le modèle booléen.

c. Enfin, écrire une fonction `evaluate_boolean_model (requests, collection_index, relevance_judgments,BooleanOperator)` qui vous permettra d'évaluer le modèle booléen sur la collection `TIME`. Pour rappel, le resultat attendu en sortie est est un dictionnaire avec comme clé l'`ID` de la requête et comme valeur un dictionnaire avec une clé par type de mesure (clées : "recall", "precision", "F_1") et en valeur le résultat de la mesure calculé sur les résultats de la requête.


In [ ]:
from Utils.Lab2 import *


def transformation_lem_query_to_boolean(query,index,operator):
    boolean_query=[]
    # A completer
    return boolean_query

def apply_boolean_model(requests, collection_index,BooleanOperator):
    results ={}
    # A completer
    return results


def evaluate_boolean_model (requests, collection_index, relevance_judgments,BooleanOperator):
    evaluation_boolean = {}
    # A completer
    return evaluation_boolean



In [ ]:
#Test

BooleanOperator = {'AND', 'NOT', 'OR'}
a = evaluate_boolean_model(lemmatized_requests, index_document, relevance_judgments,BooleanOperator)


Voici une capture d'écran d'un morceau du résultat (en considérant le cas d'une requête disjonctive) :


<img src="./Figures/boolean.png" width="600" height="600" />

### EVALUATION DE RÉSULTATS ORDONNÉS


#### Courbe Rappel-Précision


Pour les modèles qui permettent d'ordonner les résultats, nous allons d'abord les comparer à l'aide des courbes rappel-precision.


Avant de vous lancer dans cette étape, prenez le temps de relire les slides 31 à 35 du cours 4.

9- Ecrire la fonction `run_model_and_evaluate(query, query_id ,inverted_index, stats, model, relevance_judgments)` qui applique le modèle `model` à la requête et qui calcule les informations des 3 premières colonnes dans le tableau rappelé ci-dessous (cf. slide 33).


<img src="./Figures/recalprecisioncurve.png" width="500" height="500" />

In [ ]:
from Utils.Lab2 import *

collection_TIME = loadData("./Data/Time/TIME.ALL")
pre_processed_collection_TIME = collection_lemmatize(remove_stop_words(tokenize_Regexp_corpus(collection_TIME),"./Data/Time/TIME.STP"))
stats=get_stats_collection(pre_processed_collection_TIME)



def run_model_and_evaluate(query, query_id ,inverted_index, stats, model, relevance_judgments):
    # A completer 
     return result_with_relevance 

In [ ]:
#Test
query = requests["1"] # requests correspond à l'ensemble des requête récupérer à la "Deuxième étape : récupération des requêtes."
print(query)

run_model_and_evaluate(query, "1", index_frequence,stats, ['frequency','binary'],relevance_judgments)

# frequency: weighting_scheme_document (voir Lab2, modèle vectoriel)
# binary: weighting_scheme_query (voir Lab2, modèle vectoriel)

Voici une capture d'écran d'un morceau du résultat :


<img src="./Figures/test-evaluate-order.png" width="600" height="600" />


## PARTIE 2 : EXERCICES 

Une série d'exercices est disponible sur EDUNAO et est à travailler en **dehors de la séance**. Elle vous permettra de réviser pour l'examen. 